In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('../data/processed_players.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56510 entries, 0 to 56509
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   club              56510 non-null  object 
 1   age               56510 non-null  int64  
 2   position          56510 non-null  object 
 3   mins              56510 non-null  int64  
 4   goals             56510 non-null  float64
 5   assists           56510 non-null  float64
 6   motm              56510 non-null  float64
 7   rating            56510 non-null  float64
 8   league            56510 non-null  object 
 9   traded            56510 non-null  int64  
 10  w_shots           56510 non-null  float64
 11  w_yel             56510 non-null  float64
 12  w_red             56510 non-null  float64
 13  w_aerials_won     56510 non-null  float64
 14  w_tackles         56510 non-null  float64
 15  w_interceptions   56510 non-null  float64
 16  w_fouls           56510 non-null  float6

We have to encode the categorical data, I will use one hot encoding

In [4]:
# To float
to_float = df.select_dtypes(include="float64").columns

for x in to_float:
    df[x] = df[x].astype("float16")
    


In [6]:
df= pd.get_dummies(df, columns = ['club','position','league','apps_cat'])
# Delete n - 1
df = df.drop(['apps_cat_10','league_Premier League','position_MIDFIELDER','club_Wolfsburg'],axis=1)

I will use min max scaler to scale down the data but first I will split the data into train and test so the scaler of train data doesn´t affect test data.

I will use a 85-15 because we dont have a lot of data so I want to use as much as possible to train the model. Since we have an imbalanced data, we will stratify y.

In [53]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
X = df.loc[:,df.columns != 'traded'].copy()
y = df.loc[:,df.columns == 'traded'].copy()
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
X_train, X_test,y_train, y_test = train_test_split(X,y,random_state=33,test_size=0.2,shuffle=True)

In [54]:
from sklearn.preprocessing import MinMaxScaler

scaler_train = MinMaxScaler()
scaled_X_train = scaler_train.fit_transform(X_train)


scaler_test = MinMaxScaler()
scaled_X_test = scaler_test.fit_transform(X_test)

In [55]:
X_train = pd.DataFrame(scaled_X_train,columns=X_train.columns)
X_test = pd.DataFrame(scaled_X_test,columns=X_test.columns)

In [56]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)



In [57]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))



              precision    recall  f1-score   support

           0       0.77      0.88      0.82      8269
           1       0.86      0.74      0.80      8213

    accuracy                           0.81     16482
   macro avg       0.82      0.81      0.81     16482
weighted avg       0.82      0.81      0.81     16482

[[7254 1015]
 [2119 6094]]
